In [68]:
import numpy as np
import pandas as pd


In [69]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
train=pd.read_csv('/content/drive/MyDrive/Cropchallenge/CropchallengeTrain.csv')
Test_data=pd.read_csv('/content/drive/MyDrive/Cropchallenge/CropchallengeTest.csv')

In [71]:
train.head()

,Image_id,Label
0,id_02amazy34fgh2.jpg,1
1,id_02mh3w48pmyc9.jpg,0
2,id_02rpb463h9d3w.jpg,0
3,id_02wc3jeeao8ol.jpg,1
4,id_03t2hapb8wz8p.jpg,1


In [72]:
Test_data.head()

,Image_id
0,id_00exusbkgzw1b.jpg
1,id_03dqinf6w0znv.jpg
2,id_046yl0cxn3ybz.jpg
3,id_04athdtx2abyg.jpg
4,id_062aauf9e9jk0.jpg


In [73]:
images_path='/content/drive/MyDrive/Cropchallenge/Images/'

In [74]:
train['Image_id']= [images_path+i for i in train.Image_id]

In [75]:
train['Image_id']

0       /content/drive/MyDrive/Cropchallenge/Images/id...
1       /content/drive/MyDrive/Cropchallenge/Images/id...
2       /content/drive/MyDrive/Cropchallenge/Images/id...
3       /content/drive/MyDrive/Cropchallenge/Images/id...
4       /content/drive/MyDrive/Cropchallenge/Images/id...
                              ...                        
1614    /content/drive/MyDrive/Cropchallenge/Images/id...
1615    /content/drive/MyDrive/Cropchallenge/Images/id...
1616    /content/drive/MyDrive/Cropchallenge/Images/id...
1617    /content/drive/MyDrive/Cropchallenge/Images/id...
1618    /content/drive/MyDrive/Cropchallenge/Images/id...
Name: Image_id, Length: 1619, dtype: object

In [76]:
train.head()

,Image_id,Label
0,/content/drive/MyDrive/Cropchallenge/Images/id...,1
1,/content/drive/MyDrive/Cropchallenge/Images/id...,0
2,/content/drive/MyDrive/Cropchallenge/Images/id...,0
3,/content/drive/MyDrive/Cropchallenge/Images/id...,1
4,/content/drive/MyDrive/Cropchallenge/Images/id...,1


In [85]:
train['Label']=train['Label'].astype('str')

In [77]:
Test_data['Image_id']= [images_path+i for i in Test_data.Image_id]

In [78]:
Test_data

,Image_id
0,/content/drive/MyDrive/Cropchallenge/Images/id...
1,/content/drive/MyDrive/Cropchallenge/Images/id...
2,/content/drive/MyDrive/Cropchallenge/Images/id...
3,/content/drive/MyDrive/Cropchallenge/Images/id...
4,/content/drive/MyDrive/Cropchallenge/Images/id...
...,...
1075,/content/drive/MyDrive/Cropchallenge/Images/id...
1076,/content/drive/MyDrive/Cropchallenge/Images/id...
1077,/content/drive/MyDrive/Cropchallenge/Images/id...
1078,/content/drive/MyDrive/Cropchallenge/Images/id...


In [96]:
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import numpy as np
import keras
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [80]:
vgg_model = VGG19(include_top=False, input_shape=(224, 224, 3))

In [81]:
for layer in vgg_model.layers:
    layer.trainable=False
pretrained_part = Flatten()(vgg_model.layers[-1].output)

In [82]:
class_ = Dense(256, activation='relu')(pretrained_part)
output = Dense(2, activation='sigmoid')(class_)
model = Model(inputs = vgg_model.inputs, outputs = output)


In [83]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [86]:
TRAIN_GEN = ImageDataGenerator( 
    rescale=1./255, #normalisation of images
    #Augmentation
    rotation_range=40,
    shear_range=0.2,
    zoom_range =0.2,
    fill_mode='nearest',
    validation_split=0.2,
    horizontal_flip= True)

traindata = TRAIN_GEN.flow_from_dataframe(train,
                                      x_col='Image_id',
                                      y_col='Label',
                                      target_size=(224,224),
                                      class_mode='categorical',
                                      subset='training',
                                      seed=123,
                                      batch_size=32)


validationdata= TRAIN_GEN.flow_from_dataframe(train,
                                    x_col='Image_id',
                                    y_col='Label',
                                    target_size=(224,224),
                                    class_mode='categorical',
                                    subset='validation',
                                    seed=123,
                                    batch_size=32)

Test_gen=ImageDataGenerator(rescale=1./255)
testdata= Test_gen.flow_from_dataframe(Test_data,
                                    x_col='Image_id',
                                    y_col=None,
                                    target_size=(224,224),
                                    class_mode=None,
                                    seed=123,
                                    shuffle=False,
                                    batch_size=32)


                                            


Found 1296 validated image filenames belonging to 2 classes.
Found 323 validated image filenames belonging to 2 classes.
Found 1080 validated image filenames.


In [97]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=keras.metrics.AUC(name='auc'),)

In [98]:
model.fit(traindata,
          validation_data=validationdata,
          epochs=15,
          steps_per_epoch=10,
          verbose=1)

Epoch 1/15
10/10 [==============================] - 59s 6s/step - loss: 0.7130 - auc: 0.6779 - val_loss: 0.4762 - val_auc: 0.9165
Epoch 2/15
10/10 [==============================] - 44s 4s/step - loss: 0.3996 - auc: 0.9043 - val_loss: 0.3483 - val_auc: 0.9375
Epoch 4/15
10/10 [==============================] - 34s 3s/step - loss: 0.3400 - auc: 0.9328 - val_loss: 0.3306 - val_auc: 0.9358
Epoch 5/15
10/10 [==============================] - 31s 3s/step - loss: 0.2665 - auc: 0.9629 - val_loss: 0.3083 - val_auc: 0.9508
Epoch 6/15
10/10 [==============================] - 30s 3s/step - loss: 0.2371 - auc: 0.9760 - val_loss: 0.2282 - val_auc: 0.9769
Epoch 7/15
10/10 [==============================] - 28s 3s/step - loss: 0.2117 - auc: 0.9771 - val_loss: 0.2111 - val_auc: 0.9797
Epoch 8/15
10/10 [==============================] - 25s 3s/step - loss: 0.1820 - auc: 0.9849 - val_loss: 0.1833 - val_auc: 0.9870
Epoch 9/15
10/10 [==============================] - 22s 2s/step - loss: 0.1517 - auc: 0.99

In [100]:
predictions=model.predict(testdata)

In [110]:
predictions

array([[8.9139348e-01, 6.1930396e-02],
       [9.3064922e-01, 4.1474022e-02],
       [7.1575902e-05, 9.9997234e-01],
       ...,
       [9.4555813e-01, 2.1515366e-02],
       [6.3822336e-02, 9.3531579e-01],
       [1.8154952e-03, 9.9885201e-01]], dtype=float32)

In [107]:
submit=pd.read_csv('/content/drive/MyDrive/Cropchallenge/SampleSubmission .csv')

In [124]:
pred_labels=[i[1] for i in predictions]
print(pred_labels)

[0.061930396, 0.041474022, 0.99997234, 0.07787879, 0.055169504, 0.020351248, 0.9999082, 0.98374915, 0.006073675, 0.0120938085, 0.9990915, 0.9853402, 0.9893277, 0.02576334, 0.09682238, 0.99840325, 0.99945265, 0.99894077, 0.07770824, 0.036746457, 0.47310683, 0.03275901, 0.06826741, 0.022823539, 0.9996574, 0.0059258086, 0.9998591, 0.017734438, 0.024310961, 0.020844033, 0.1373625, 0.027572071, 0.8309295, 0.09922882, 0.0049644993, 0.99973387, 0.0070528765, 0.99996734, 0.99987483, 0.9999732, 0.0073536085, 0.9992593, 0.2802293, 0.99963534, 0.9995919, 0.99940777, 0.9998857, 0.99976915, 0.79325527, 0.9995989, 0.9826125, 0.8346401, 0.9994518, 0.029769436, 0.9544776, 0.99906224, 0.9967019, 0.99995804, 0.91700315, 0.9993542, 0.9881293, 0.70885986, 0.965872, 0.010045723, 0.9303301, 0.031565316, 0.9992879, 0.008521521, 0.30318338, 0.9991097, 0.16170299, 0.013059495, 0.99964905, 0.99715877, 0.98310983, 0.48157495, 0.99493164, 0.026421098, 0.017436119, 0.9826095, 0.06088507, 0.9965485, 0.010889773, 0.

In [126]:
submission = pd.DataFrame({ 'Image_id':submit.Image_id.values, 'Label': pred_labels})
submission.to_csv("my_submission.csv", index=False)